In [125]:
import pandas as pd
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import numpy as np

In [126]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

In [127]:
RSE = pd.read_csv("road2.csv", encoding='cp949')
RSE.head()

,노선번호,도로명,이정,X좌표값,Y좌표값,GRS80X좌표값,GRS80Y좌표값
0,10,경부선,0.0,35.246578,129.093207,390524.290125,296459.942624
1,10,경부선,0.1,35.247249,129.093941,390589.576852,296535.747996
2,10,경부선,0.2,35.247919,129.094676,390654.863578,296611.553368
3,10,경부선,0.3,35.248590,129.095410,390720.150305,296687.358740
4,10,경부선,0.4,35.249260,129.096144,390785.437032,296763.164111


In [128]:
RSE.tail()

,노선번호,도로명,이정,X좌표값,Y좌표값,GRS80X좌표값,GRS80Y좌표값
45949,6000,부산외곽순환선,50.1,35.284754,129.228807,402773.478106,300965.275385
45950,6000,부산외곽순환선,50.2,35.285630,129.229062,402794.523435,301062.975344
45951,6000,부산외곽순환선,50.3,35.286513,129.229282,402812.316043,301161.379729
45952,6000,부산외곽순환선,50.4,35.287396,129.229494,402829.419219,301259.899480
45953,6000,부산외곽순환선,50.5,35.288092,129.229630,402840.037096,301337.359488


In [129]:
yeongdong_line = RSE[RSE.노선번호 == 500]
yeongdong_line.head()

,노선번호,도로명,이정,X좌표값,Y좌표값,GRS80X좌표값,GRS80Y좌표값
31263,500,영동선,0.0,37.437852,126.739665,176960.712022,537638.347773
31264,500,영동선,0.1,37.436968,126.739448,176941.216384,537540.269249
31265,500,영동선,0.2,37.436084,126.739227,176921.353439,537442.264490
31266,500,영동선,0.3,37.435201,126.739004,176901.330143,537344.291504
31267,500,영동선,0.4,37.434317,126.738787,176881.892606,537246.199624


In [130]:
seoul_line = RSE[RSE.노선번호 == 1000]
seoul_line.head()

,노선번호,도로명,이정,X좌표값,Y좌표값,GRS80X좌표값,GRS80Y좌표값
40414,1000,수도권제1순환선,0.0,37.405752,127.094647,208379.667272,534048.063192
40415,1000,수도권제1순환선,0.1,37.406091,127.095693,208472.300625,534085.810181
40416,1000,수도권제1순환선,0.2,37.406430,127.096740,208564.933879,534123.557270
40417,1000,수도권제1순환선,0.3,37.406768,127.097787,208657.636934,534161.131862
40418,1000,수도권제1순환선,0.4,37.407073,127.098850,208751.702337,534195.089905


In [131]:
gyungbu_line = RSE[RSE.노선번호 == 10]
gyungbu_line.head()

,노선번호,도로명,이정,X좌표값,Y좌표값,GRS80X좌표값,GRS80Y좌표값
0,10,경부선,0.0,35.246578,129.093207,390524.290125,296459.942624
1,10,경부선,0.1,35.247249,129.093941,390589.576852,296535.747996
2,10,경부선,0.2,35.247919,129.094676,390654.863578,296611.553368
3,10,경부선,0.3,35.248590,129.095410,390720.150305,296687.358740
4,10,경부선,0.4,35.249260,129.096144,390785.437032,296763.164111


In [132]:
Top = pd.concat([yeongdong_line, seoul_line, gyungbu_line], axis = 0)

In [146]:
Top = Top[Top['이정']%1 == 0]

In [159]:
m_1 = folium.Map(location=[37.2,127], tiles='cartodbpositron', zoom_start=9)

def color_producer(val, road):
    if road == 10 :
        if val >= 392.525 and val <= 397.75:
            return 'red'
        elif val >= 400.575 and val <= 413.8 :
            return 'red'
        else:
            return 'forestgreen'
    elif road == 1000 :
        if val >= 0 and val <= 4.2 :
            return 'red'
        elif val >= 9.5 and val <= 14.5 :
            return 'red'
        elif val >= 19.5 and val <= 22.1 :
            return 'red'
        elif val >= 22.1 and val <= 25.1 :
            return 'red'
        elif val >= 72.1 and val <= 75.2 :
            return 'red'
        elif val >= 75.2 and val <= 79 :
            return 'orange'
        elif val >= 82 and val <= 90.8 :
            return 'red'
        else :
            return 'forestgreen'
    elif road == 500 :
        if val >= 25.9 and val <= 27.9 :
            return 'red'
        else :
            return 'forestgreen'
        
m_1 = add_categorical_legend(m_1, '1st Year',
                             colors = ['#000', '#FF0000', '#FF7F00', '#228B22'],
                           labels = ['Complete',  'Progressing', 'Carry Forward', 'None'])

# 도로이정 값을 기준으로 100 이하는 초록색, 그외는 빨강색으로 추가했습니다.
for i in range(0,len(Top)):
    Circle(
        location=[Top.iloc[i]['X좌표값'], Top.iloc[i]['Y좌표값']],
        radius=20,
        color=color_producer(Top.iloc[i]['이정'],Top.iloc[i]['노선번호'] )).add_to(m_1)

In [160]:
m_2 = folium.Map(location=[37.2,127], tiles='cartodbpositron', zoom_start=9)

def color_producer(val, road):
    if road == 10 :
        if val >= 392.525 and val <= 397.75:
            return 'black'
        elif val >= 400.575 and val <= 413.8 :
            return 'black'
        elif val >= 344.675 and val <= 353 :
            return 'red'
        elif val >= 364.925 and val <= 369.85 :
            return 'red'
        elif val >= 383.275 and val <= 387.3 :
            return 'red'        
        else:
            return 'forestgreen'
    elif road == 1000 :
        if val >= 0 and val <= 4.2 :
            return 'black'
        elif val >= 9.5 and val <= 14.5 :
            return 'black'
        elif val >= 19.5 and val <= 22.1 :
            return 'black'
        elif val >= 22.1 and val <= 25.1 :
            return 'black'
        elif val >= 72.1 and val <= 75.2 :
            return 'black'
        elif val >= 75.2 and val <= 79 :
            return 'red'
        elif val >= 82 and val <= 90.8 :
            return 'black'
        elif val >= 4.2 and val <= 9.5 :
            return 'red'
        elif val >= 14.5 and val <= 19.5 :
            return 'red'
        elif val >= 26.4 and val <= 28.8 :
            return 'red'
        elif val >= 28.8 and val <= 30.8 :
            return 'orange'
        elif val >= 79 and val <= 82 :
            return 'red'
        elif val >= 43.6 and val <= 45.2 :
            return 'red'
        else :
            return 'forestgreen'
    elif road == 500 :
        if val >= 25.9 and val <= 27.9 :
            return 'black'
        elif val >= 0 and val <= 5.9 :
            return 'red'
        elif val >= 12.4 and val <= 15.7 :
            return 'red'
        elif val >= 22.6 and val <= 25.95 :
            return 'red'
        elif val >= 27.9 and val <= 30.7 :
            return 'red'
        else :
            return 'forestgreen'

m_2 = add_categorical_legend(m_2, '2nd Year',
                             colors = ['#000', '#FF0000', '#FF7F00', '#228B22'],
                           labels = ['Complete',  'Progressing', 'Carry Forward', 'None'])
        
# 도로이정 값을 기준으로 100 이하는 초록색, 그외는 빨강색으로 추가했습니다.
for i in range(0,len(Top)):
    Circle(
        location=[Top.iloc[i]['X좌표값'], Top.iloc[i]['Y좌표값']],
        radius=20,
        color=color_producer(Top.iloc[i]['이정'],Top.iloc[i]['노선번호'] )).add_to(m_2)

In [161]:
m_3 = folium.Map(location=[37.2,127], tiles='cartodbpositron', zoom_start=9)

def color_producer(val, road):
    if road == 10 :
        if val >= 392.525 and val <= 397.75:
            return 'black'
        elif val >= 400.575 and val <= 413.8 :
            return 'black'
        elif val >= 344.675 and val <= 353 :
            return 'black'
        elif val >= 364.925 and val <= 369.85 :
            return 'black'
        elif val >= 383.275 and val <= 387.3 :
            return 'black'
        elif val >= 353 and val <= 364.925 :
            return 'orange'
        elif val >= 369.85 and val <= 383.275 :
            return 'red'
        elif val >= 387.3 and val <= 392.525 :
            return 'red'
        elif val >= 397.75 and val <= 400.575 :
            return 'red'
        else:
            return 'forestgreen'
    elif road == 1000 :
        if val >= 0 and val <= 4.2 :
            return 'black'
        elif val >= 9.5 and val <= 14.5 :
            return 'black'
        elif val >= 19.5 and val <= 22.1 :
            return 'black'
        elif val >= 22.1 and val <= 25.1 :
            return 'black'
        elif val >= 72.1 and val <= 75.2 :
            return 'black'
        elif val >= 75.2 and val <= 79 :
            return 'black'
        elif val >= 82 and val <= 90.8 :
            return 'black'
        elif val >= 4.2 and val <= 9.5 :
            return 'black'
        elif val >= 14.5 and val <= 19.5 :
            return 'black'
        elif val >= 26.4 and val <= 28.8 :
            return 'black'
        elif val >= 28.8 and val <= 30.8 :
            return 'red'
        elif val >= 79 and val <= 82 :
            return 'black'
        elif val >= 43.6 and val <= 45.2 :
            return 'black'
        elif val >= 30.8 and val <= 33.6 :
            return 'red'
        elif val >= 102.5 and val <= 106.9 :
            return 'red'
        elif val >= 119.2 and val <= 128 :
            return 'red'
        elif val >= 66.6 and val <= 69.9 :
            return 'red'
        else :
            return 'forestgreen'
    elif road == 500 :
        if val >= 25.9 and val <= 27.9 :
            return 'black'
        elif val >= 0 and val <= 5.9 :
            return 'black'
        elif val >= 12.4 and val <= 15.7 :
            return 'black'
        elif val >= 22.6 and val <= 25.95 :
            return 'black'
        elif val >= 27.9 and val <= 30.7 :
            return 'black'
        elif val >= 17.8 and val <= 22.6 :
            return 'red'
        else :
            return 'forestgreen'

m_3 = add_categorical_legend(m_3, '3rd Year',
                             colors = ['#000', '#FF0000', '#FF7F00', '#228B22'],
                           labels = ['Complete',  'Progressing', 'Carry Forward', 'None'])

# 도로이정 값을 기준으로 100 이하는 초록색, 그외는 빨강색으로 추가했습니다.
for i in range(0,len(Top)):
    Circle(
        location=[Top.iloc[i]['X좌표값'], Top.iloc[i]['Y좌표값']],
        radius=20,
        color=color_producer(Top.iloc[i]['이정'],Top.iloc[i]['노선번호'] )).add_to(m_3)

In [162]:
m_4 = folium.Map(location=[37.2,127], tiles='cartodbpositron', zoom_start=9)

def color_producer(val, road):
    if road == 10 :
        if val >= 392.525 and val <= 397.75:
            return 'black'
        elif val >= 400.575 and val <= 413.8 :
            return 'black'
        elif val >= 344.675 and val <= 353 :
            return 'black'
        elif val >= 364.925 and val <= 369.85 :
            return 'black'
        elif val >= 383.275 and val <= 387.3 :
            return 'black'
        elif val >= 353 and val <= 364.925 :
            return 'red'
        elif val >= 369.85 and val <= 383.275 :
            return 'black'
        elif val >= 387.3 and val <= 392.525 :
            return 'black'
        elif val >= 397.75 and val <= 400.575 :
            return 'black'
        elif val >= 337.85 and val <= 344.675 :
            return 'red'
        else:
            return 'forestgreen'
    elif road == 1000 :
        if val >= 0 and val <= 4.2 :
            return 'black'
        elif val >= 9.5 and val <= 14.5 :
            return 'black'
        elif val >= 19.5 and val <= 22.1 :
            return 'black'
        elif val >= 22.1 and val <= 25.1 :
            return 'black'
        elif val >= 72.1 and val <= 75.2 :
            return 'black'
        elif val >= 75.2 and val <= 79 :
            return 'black'
        elif val >= 82 and val <= 90.8 :
            return 'black'
        elif val >= 4.2 and val <= 9.5 :
            return 'black'
        elif val >= 14.5 and val <= 19.5 :
            return 'black'
        elif val >= 26.4 and val <= 28.8 :
            return 'black'
        elif val >= 28.8 and val <= 30.8 :
            return 'black'
        elif val >= 79 and val <= 82 :
            return 'black'
        elif val >= 43.6 and val <= 45.2 :
            return 'black'
        elif val >= 30.8 and val <= 33.6 :
            return 'black'
        elif val >= 102.5 and val <= 106.9 :
            return 'black'
        elif val >= 119.2 and val <= 128 :
            return 'black'
        elif val >= 66.6 and val <= 69.9 :
            return 'black'
        elif val >= 69.9 and val <= 72.1 :
            return 'red'
        elif val >= 90.8 and val <= 97.7 :
            return 'red'
        elif val >= 106.9 and val <= 112.2 :
            return 'red'
        elif val >= 116.3 and val <= 119.2 :
            return 'red'
        elif val >= 45.2 and val <= 51.7 :
            return 'red'
        elif val >= 51.7 and val <= 60.6 :
            return 'orange'
        else :
            return 'forestgreen'
    elif road == 500 :
        if val >= 25.9 and val <= 27.9 :
            return 'black'
        elif val >= 0 and val <= 5.9 :
            return 'black'
        elif val >= 12.4 and val <= 15.7 :
            return 'black'
        elif val >= 22.6 and val <= 25.95 :
            return 'black'
        elif val >= 27.9 and val <= 30.7 :
            return 'black'
        elif val >= 17.8 and val <= 22.6 :
            return 'black'
        else :
            return 'forestgreen'

m_4 = add_categorical_legend(m_4, '4th Year',
                             colors = ['#000', '#FF0000', '#FF7F00', '#228B22'],
                           labels = ['Complete',  'Progressing', 'Carry Forward', 'None'])

# 도로이정 값을 기준으로 100 이하는 초록색, 그외는 빨강색으로 추가했습니다.
for i in range(0,len(Top)):
    Circle(
        location=[Top.iloc[i]['X좌표값'], Top.iloc[i]['Y좌표값']],
        radius=20,
        color=color_producer(Top.iloc[i]['이정'],Top.iloc[i]['노선번호'] )).add_to(m_4)

In [163]:
m_5 = folium.Map(location=[37.2,127], tiles='cartodbpositron', zoom_start=9)

def color_producer(val, road):
    if road == 10 :
        if val >= 392.525 and val <= 397.75:
            return 'black'
        elif val >= 400.575 and val <= 413.8 :
            return 'black'
        elif val >= 344.675 and val <= 353 :
            return 'black'
        elif val >= 364.925 and val <= 369.85 :
            return 'black'
        elif val >= 383.275 and val <= 387.3 :
            return 'black'
        elif val >= 353 and val <= 364.925 :
            return 'black'
        elif val >= 369.85 and val <= 383.275 :
            return 'black'
        elif val >= 387.3 and val <= 392.525 :
            return 'black'
        elif val >= 397.75 and val <= 400.575 :
            return 'black'
        elif val >= 337.85 and val <= 344.675 :
            return 'black'
        elif val >= 135.725 and val <= 141.35 :
            return 'red'
        elif val >= 415.225 and val <= 422.15 :
            return 'red'
        else:
            return 'forestgreen'
    elif road == 1000 :
        if val >= 0 and val <= 4.2 :
            return 'black'
        elif val >= 9.5 and val <= 14.5 :
            return 'black'
        elif val >= 19.5 and val <= 22.1 :
            return 'black'
        elif val >= 22.1 and val <= 25.1 :
            return 'black'
        elif val >= 72.1 and val <= 75.2 :
            return 'black'
        elif val >= 75.2 and val <= 79 :
            return 'black'
        elif val >= 82 and val <= 90.8 :
            return 'black'
        elif val >= 4.2 and val <= 9.5 :
            return 'black'
        elif val >= 14.5 and val <= 19.5 :
            return 'black'
        elif val >= 26.4 and val <= 28.8 :
            return 'black'
        elif val >= 28.8 and val <= 30.8 :
            return 'black'
        elif val >= 79 and val <= 82 :
            return 'black'
        elif val >= 43.6 and val <= 45.2 :
            return 'black'
        elif val >= 30.8 and val <= 33.6 :
            return 'black'
        elif val >= 102.5 and val <= 106.9 :
            return 'black'
        elif val >= 119.2 and val <= 128 :
            return 'black'
        elif val >= 66.6 and val <= 69.9 :
            return 'black'
        elif val >= 69.9 and val <= 72.1 :
            return 'black'
        elif val >= 90.8 and val <= 97.7 :
            return 'black'
        elif val >= 106.9 and val <= 112.2 :
            return 'black'
        elif val >= 116.3 and val <= 119.2 :
            return 'black'
        elif val >= 45.2 and val <= 51.7 :
            return 'black'
        elif val >= 51.7 and val <= 60.6 :
            return 'red'
        elif val >= 25.1 and val <= 26.4 :
            return 'red'
        elif val >= 97.7 and val <= 102.5 :
            return 'red'
        elif val >= 112.2 and val <= 116.3 :
            return 'orange'
        elif val >= 35.6 and val <= 43.6 :
            return 'red'
        elif val >= 60.6 and val <= 66.6 :
            return 'red'
        else :
            return 'forestgreen'
    elif road == 500 :
        if val >= 25.9 and val <= 27.9 :
            return 'black'
        elif val >= 0 and val <= 5.9 :
            return 'black'
        elif val >= 12.4 and val <= 15.7 :
            return 'black'
        elif val >= 22.6 and val <= 25.95 :
            return 'black'
        elif val >= 27.9 and val <= 30.7 :
            return 'black'
        elif val >= 17.8 and val <= 22.6 :
            return 'black'
        elif val >= 30.7 and val <= 41.7 :
            return 'red'
        else :
            return 'forestgreen'

m_5 = add_categorical_legend(m_5, '5th Year',
                             colors = ['#000', '#FF0000', '#FF7F00', '#228B22'],
                           labels = ['Complete',  'Progressing', 'Carry Forward', 'None'])

# 도로이정 값을 기준으로 100 이하는 초록색, 그외는 빨강색으로 추가했습니다.
for i in range(0,len(Top)):
    Circle(
        location=[Top.iloc[i]['X좌표값'], Top.iloc[i]['Y좌표값']],
        radius=20,
        color=color_producer(Top.iloc[i]['이정'],Top.iloc[i]['노선번호'] )).add_to(m_5)

In [164]:
m_1

In [165]:
m_2

In [166]:
m_3

In [167]:
m_4

In [168]:
m_5